In [1]:
import os
import sys
os.chdir("/home/nile")
os.curdir
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME']='/home/nile/spark-2.1.0-bin-hadoop2.7'
SPARK_HOME=os.environ['SPARK_HOME']

sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.4-src.zip"))

from pyspark import SparkContext
from pyspark import SparkConf

conf=SparkConf()
conf.set("spark.executor.memory","1g")
conf.set("spark.cores.max","1")

conf.setAppName("major")

sc = SparkContext('local',conf=conf)


In [2]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
import nltk.corpus
import string
import re , math
from collections import Counter

In [3]:
f = open('slang.txt', 'r')
slang_dict = {}
for line in f:
    line = line.strip()
    parts = [p.strip() for p in line.split("=")]
    slang_dict[parts[0].lower()] = (parts[1].lower())
f.close()

In [4]:
rdd1 = sc.textFile("/home/nile/drive/major/major2/codes/combined_new.txt")
rdd1.count()

9582

In [5]:
def preprocess(x):
    l = x.split(',')
    if(len(l)>=4):
        #convert to lower case
        text = str(l[3].decode('utf-8')).lower()
        text=text.replace('"text":', '')

    else:
        text  = "not there"
        
    return(Row(tweet=text))
        
        

In [6]:
rdd1=rdd1.map(preprocess)
rdd1.count()
rdd1.collect()

[Row(tweet='"@nyctsubway do these work better and not bend!? #sarcasm"'),
 Row(tweet='not there'),
 Row(tweet='"about to get my gums lasered on. oh'),
 Row(tweet='not there'),
 Row(tweet='"must be so.\\na \\"swede\\" is just a primitive'),
 Row(tweet='not there'),
 Row(tweet='"no groundhog for groundhog\'s day. no valentine for valentine\'s day. no president for president\'s day. #notmypresident #sarcasm"'),
 Row(tweet='not there'),
 Row(tweet='"@jeffreyguterman @realdonaldtrump as opposed to the successful efforts to end the killing by the #obama #hrc #kerry regime? #sarcasm"'),
 Row(tweet='not there'),
 Row(tweet='"now'),
 Row(tweet='not there'),
 Row(tweet='"@nfltotalaccess @juliojones_11 @millerlite40 @l_bell26 i would put @rgiii. #nfl #sarcasm"'),
 Row(tweet='not there'),
 Row(tweet='"@randalusa @morningmika wow- randal here sounds like a really smart guy'),
 Row(tweet='not there'),
 Row(tweet='"in jail i ask my mom for a meal that i don\'t get. my wife goes in and my mom is plann

In [7]:
def filter_trainingSet(x):
    li = x[0].split(' ')
    if "like" in li:
        return x

In [8]:
rdd2=rdd1.filter(filter_trainingSet)
rdd2.count()

254

In [9]:
def preprocess_training(x):
    text = x[0]
    l1 = text.split(" ")
    if '' in l1:
        l1.remove('')
    for i in range(len(l1)):
        if l1[i] in slang_dict:
            l1[i] = slang_dict[l1[i]]
    text=' '.join(l1)

    #covert @user to at_user
    text= re.sub('@[^\s]+','atuser',text)

    #covert #data to data
    #text = re.sub(r'#([^\s]+)', r'\1', text)

    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)

    text  = text.replace('\\', '')
    #replace url by 'url'
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)

     #--feature vector--#

    #1)removing punctuation marks
    text =  text.translate(string.maketrans("",""), string.punctuation)

     #3) removing repeated chars
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    text = pattern.sub(r"\1\1", text)

    #3) removing words containing no
    text=re.sub(r'\w*\d\w*', '', text).strip()
    return(Row(tweet=text))

In [10]:
rdd2 = rdd2.map(preprocess_training)
rdd2.collect()


[Row(tweet='atuser atuser wow randal here sounds like a really smart guy'),
 Row(tweet='lol funny comedy sarcasm youre like a coin aww'),
 Row(tweet='lol funny comedy sarcasm get you a man like this url'),
 Row(tweet='atuser notmypresidentsday lol thats funny kinda like a play on words'),
 Row(tweet='atuser how dare you guys want a day off tell like you do anything all day long sarcasm'),
 Row(tweet='theres nothing like being racially profiled in barnes and noble to make you feel valued sarcasm'),
 Row(tweet='nioh connection error when trying to use the tori gate ah just like darksouls failed to summon good times sarcasm'),
 Row(tweet='atuser so  were supposed to like if hillary is not a fascist man child sarcasm'),
 Row(tweet='atuser atuser atuser because nothing says stability like complete instability good one atuser leadership sarcasm auspol'),
 Row(tweet='sar called me an instahoe in a comment on my insta amp like honestly its just because shes jealous of my insta jealous sarcasm'

In [11]:
rdd1 = rdd1.map(preprocess_training)
rdd1.count()

9582

In [12]:
def filter_null(data):
    text = data[0]
    if text != "not there":
        return data

In [13]:
rdd1= rdd1.filter(filter_null)
rdd1.count()

4791

In [14]:
import enchant
import editdistance 
d = enchant.Dict("en_US")

In [15]:
def proc(x):
    tweet = str(x[0])
    #tweet = tweet.split(" ")
    
    #spell correction
    skip = True
    suggestion_list = []
    distances = []
    data = []
    words=tweet.split(" ")
    for k in range(len(words)):
        if ' ' in words :
            words.remove(' ')
    for k in range(len(words)):
        if  words[k]!='':
            if words[k] == 'url':
                pass
            if(d.check(words[k])==False):
                suggestion=d.suggest(words[k])
                if len(suggestion)!=0:
                    for i in range(0,len(suggestion)):
                        distance = editdistance.eval(words[k],suggestion[i])
                        distances.append(distance)
                        suggestion_list.append(suggestion[i])
                    index = distances.index(min(distances))
                    words[k]=suggestion_list[index]
        distances = []
        suggestion_list = []
    tweet=" ".join(words)
    return Row(tweet=str(tweet))


In [16]:
rdd2 = rdd2.map(proc)
rdd2.count()

254

In [17]:
##COLLECTING NEGATIVE SITUATIONS

In [18]:
negative = list() #this list will contain all thenegative situations

In [19]:
def calc_freq(rdd1,gram):
    count = 0
    for x in rdd1.collect():
        tweet = x[0]
        if gram in tweet:
            count = count+1
            

In [20]:
def calc_score(x):
    count_neutral = calc_freq(neutral_rdd,x)
    count_sarcastic = calc_freq(final,x)
    score = count_sarcastic/(count_neutral+count_sarcastic)
    return score

In [21]:
##for one gram
def check_1gram(text,target_list):
    part=[]
    
    l = nltk.word_tokenize(text)
    pos = nltk.pos_tag(l)
    
    for li in pos:
        #print li
        if li[1]=='VB' or li[1]=='VBD' or li[1]=='VBG' or li[1]=='VBN' or li[1]=='VBP' or li[1]=='VBN' or li[1]=='NN':
            #print text
            target_list.append(text)
            #print score(text)
            #we node to compute the score which will decide weather to append it in list


In [22]:
## for 2 gram and 3 gram
comb_2 = [['V','ADJ'], ['V','V'], ['V','ADV'], ['ADV','V'], ['ADV','NOUN'],['V','NOUN'], ['V','PRO'], ['to','V'],['ADV','ADJ'],['PRO','V']]
comb_3 = [['ADV','V','NOUN'],['V','V','V'],['to','V','V'],['V','CC','V'],['to', 'V', 'NOUN'], ['to', 'V', 'ADJ'], ['PRO', 'ADJ', 'NOUN'], ['V', 'to', 'NOUN'], ['to', 'V', 'ADV'], ['V', 'ADJ', 'NOUN']]

def check_ngram(text,list_type,target_list):
    #print text
    part=[]
    l1 = list()
    l = nltk.word_tokenize(text)
    pos = nltk.pos_tag(l)
    for li in pos:
        if li[1]=='VB' or li[1]=='VBD' or li[1]=='VBG' or li[1]=='VBN' or li[1]=='VBP' or li[1]=='VBN':
            l1.append('V')
        elif li[1]=='NN' or li[1]=='NNS' or li[1]=='NNP' or li[1]=='NNPS': 
            l1.append('NOUN')
        elif li[1]=='RB' or li[1]=='RBR' or li[1]=='RBS':
            l1.append('ADV')
        elif li[1]=="TO":
            l1.append('to')
        elif li[1]=="CC":
            l1.append("CC")
        elif li[1]=='JJ' or li[1]=='JJR' or li[1]=='JJS':
            l1.append("ADJ")
        elif li[1]=='PRP' or li[1]=='PRP$' or li[1]=='WP$':
            l1.append("PRO")
        
    for li in list_type:
        if li == l1:
            print text
            target_list.append(text)
            #print calc_score(text)
            #we node to compute the score which will decide weather to append it in list
        


In [23]:
def extract_grams(tweet):
    words = tweet.split(" ")
    grams = []
    for i in range(len(words)):
        if words[i] == "like":
            k = i+1
            for j in range(k+1,k+4):
                 grams.append((' ').join(words[k:j]))
    for term in grams:
        if len(term.split(' '))==1:
            check_1gram(term, negative)
        elif len(term.split(' '))==2:
            check_ngram(term,comb_2, negative)
        elif len(term.split(' '))==3:
            check_ngram(term,comb_3, negative)

In [24]:
for x in rdd2.collect():
    tweet = x[0]
    extract_grams(tweet)

you do
being racially
doing open
doing open source
we have
we have
talking to humans
we proved
to done
to done i
to chill
to chill
wed be
to have
to have sex
linking park
them sarcasm
them sarcasm
you Ni
him i
him getting
my first post
to do
you work
to thank
to thank
me please
me please


In [25]:
negative =  list(set(negative))
print negative

['being racially', 'to done i', 'being', 'my first post', 'doing open source', 'cheap', 'linking', 'talking', 'them sarcasm', 'doing open', 'to done', 'shit', 'trump', 'Benghazi', 'Bangkok', 'asthma', 'wee', 'wed', 'wonderer', 'doing', 'talking to humans', 'progress', 'reading', 'linking park', 'grok', 'him i', 'newton', 'Justin', 'to do', 'him getting', 'boobs', 'Ur', 'dark', 'overcrowded', 'jag', 'date', 'pk', 'wed be', 'to thank', 'Ive', 'you Ni', 'language', 'you do', 'i', 'stalwart', 'we proved', 'me please', 'um', 'to chill', 'we have', 'to have', 'to have sex', 'chemistry', 'you work']


In [26]:
f = open("/home/nile/drive/major/major2/codes/negative.txt",'a')
for item in negative:
     f.write("%s\n" % item)

In [27]:
###extracting positive situations

In [28]:
positive = list()

In [29]:
def extract_pos_grams(tweet_list,index):
   
    if(index>=2):
        start = index - 2
    else:
        start = index - 1
    pos_grams = list()
    for i in range(start,index):
        pos_grams.append((' ').join(tweet_list[i:index]))
    
    for term in pos_grams:
        
        if len(term.split(' '))==1:
            check_1gram(term,positive)
        if len(term.split(' '))==2:
            check_ngram(term,comb_2,positive)
        
        
    
    

In [30]:
for x in rdd1.collect():
    tweet = x[0]
    li = tweet.split(' ')
    for index in range(len(li)):
        if li[index] in negative:
            extract_pos_grams(li,index)

atuser atuser
atuser atuser
just realized
are you
hed be
hed be
hed be
atuser atuser
hed be
atuser atuser
atuser atuser
to leave
we impeach
so glad
tried not
be spot
did not
atuser atuser
only thing
atuser atuser
atuser atuser
atuser atuser
atuser atuser
to say
they took
atuser atuser
atuser atuser
atuser atuser
to make
to donme
atuser atuser
too bad
coming upbut
coming upbut
they are
they are
you thought
atuser atuser
be afraid
back send
atuser atuser
so hard
atuser atuser
atuser atuser
atuser yes
you guys
so glad
said ok
atuser ok
are legitimately
to whomever
atuser atuser
atuser atuser
first month
do love
never ordered
never ordered
atuser atuser
atuser atuser
telling me
shocked shocked
us know
you figure
bully stop
atuser atuser
atuser atuser
me wow
are you
shocked shocked
wotaking bribes
to stop
hearing your
so glad
so shocking
we like
going rates
going rates
atuser atuser
atuser atuser
only reason
love it
atuser atuser
we r
you mean
you mean
testing great
just wish
just wish
to a

In [31]:
print positive

['atuser atuser', 'atuser', 'jail', 'donald', 'care', 'youre', 'swear', 'nso', 'ha', 'gee', 'question', 'noted', 'noted', 'noted', 'noted', 'noted', 'donald', 'bahahahaha', 'noted', 'noted', 'atuser', 'noted', 'atuser atuser', 'atuser', 'just realized', 'realized', 'person', 'are you', 'hed be', 'be', 'hed be', 'be', 'hed be', 'be', 'atuser', 'sarcasm', 'wish', 'holy', 'holy', 'atuser atuser', 'atuser', 'hed be', 'be', 'keep', 'atuser', 'amp', 'atuser atuser', 'atuser', 'atuser atuser', 'atuser', 'sarcasm', 'man', 'to leave', 'leave', 'we impeach', 'impeach', 'so glad', 'glad', 'dont', 'breathe', 'tried not', 'atuser', 'oh', 'oh', 'do', 'atuser', 'be spot', 'spot', 'im', 'service', 'atuser', 'did not', 'atuser', 'atuser', 'sarcasm', 'didnt', 'oh', 'word', 'damn', 'atuser', 'atuser atuser', 'atuser', 'oh', 'atuser', 'atuser', 'agree', 'only thing', 'thing', 'atuser atuser', 'atuser', 'grammys', 'atuser', 'atuser', 'atuser', 'atuser', 'nnpoll', 'atuser', 'atuser', 'atuser', 'thx', 'reaso

In [32]:
positive = list(set(positive))
positive.append('like')
print positive

['cute', 'jerkam', 'to whomever', 'being', 'money', 'weekend', 'month', 'country', 'course', 'to donme', 'amp', 'upbut', 'fine', 'funfact', 'wage', 'so shocking', 'only thing', 'writer', 'send', 'going', 'coming upbut', 'isaac', 'going rates', 'worth', 'meant', 'do', 'to leave', 'stop', 'hate', 'are legitimately', 'umm', 'game', 'know', 'gee', 'press', 'world', 'only reason', 'cool', 'just realized', 'russia', 'everytime', 'did', 'radiant', 'list', 'you figure', 'noted', 'leave', 'stuff', 'back send', 'atuser ok', 'they took', 'fault', 'wotaking bribes', 'side', 'mean', 'requirement', 'everyone', 'to say', 'twitter', 'idea', 'impeach', 'calling', 'aggravated', 'donald', 'are', 'follow', 'morning', 'hearing your', 'are you', 'humor', 'mood', 'horrid', 'matter', 'whaat', 'god', 'wow', 'nnbut', 'race', 'health', 'whomever', 'presidentbusinessman', 'cnn', 'reply', 'cause', 'be', 'reporting', 'omg', 'libtard', 'we like', 'workflow', 'libs', 'ltskermit', 'guess', 'cork', 'reason', 'devout', 

In [34]:
f = open("/home/nile/drive/major/major2/codes/positive.txt",'a')
for item in positive:
     f.write("%s\n" % item)